In [ ]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple 

     ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━ 1.0/2.3 MB 14.4 kB/s eta 0:01:32

## 3. 观点抽取
### 3.0 载入模型和Tokenizer

In [2]:
import paddlenlp
from paddlenlp.transformers import SkepForTokenClassification, SkepTokenizer

### 3.1 数据处理

In [3]:
# 解压数据
!unzip -o datasets/COTE-BD
!unzip -o datasets/COTE-DP
!unzip -o datasets/COTE-MFW

unzip:  cannot find or open datasets/COTE-BD, datasets/COTE-BD.zip or datasets/COTE-BD.ZIP.
unzip:  cannot find or open datasets/COTE-DP, datasets/COTE-DP.zip or datasets/COTE-DP.ZIP.
unzip:  cannot find or open datasets/COTE-MFW, datasets/COTE-MFW.zip or datasets/COTE-MFW.ZIP.


数据内部结构解析（三个数据集的结构相同）：
```
train:
label		text_a
鸟人		《鸟人》一书以鸟博士的遭遇作为主线，主要写了鸟博士从校园出来后的种种荒诞经历。
...		...
test:
qid		text_a
0		毕棚沟的风景早有所闻，尤其以秋季的风景最美，但是这次去晚了，红叶全掉完了，黄叶也看不到了，下了雪只...
...		...

In [4]:
# 得到数据集字典
def open_func(file_path):
    return [line.strip() for line in open(file_path, 'r', encoding='utf8').readlines()[1:] if len(line.strip().split('\t')) >= 2]

data_dict = {'cotebd': {'test': open_func('COTE-BD/test.tsv'),
                        'train': open_func('COTE-BD/train.tsv')},
             'cotedp': {'test': open_func('COTE-DP/test.tsv'),
                        'train': open_func('COTE-DP/train.tsv')},
             'cotemfw': {'test': open_func('COTE-MFW/test.tsv'),
                        'train': open_func('COTE-MFW/train.tsv')}}

### 3.2 定义数据读取器
思路类似，需要注意的是这一次是Tokens级的分类。在数据读取器中，将label写成BIO的形式，每一个token都对应一个label。

In [5]:
# 定义数据集
from paddle.io import Dataset, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
import numpy as np
label_list = {'B': 0, 'I': 1, 'O': 2}
index2label = {0: 'B', 1: 'I', 2: 'O'}

# 考虑token_type_id
class MyDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512, for_test=False):
        super().__init__()
        self._data = data
        self._tokenizer = tokenizer
        self._max_len = max_len
        self._for_test = for_test
    
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, idx):
        samples = self._data[idx].split('\t')
        label = samples[-2]
        text = samples[-1]
        if self._for_test:
            origin_enc = self._tokenizer.encode(text, max_seq_len=self._max_len)['input_ids']
            return np.array(origin_enc, dtype='int64')
        else:
            
            # 由于并不是每个字都是一个token，这里采用一种简单的处理方法，先编码label，再编码text中除了label以外的词，最后合到一起
            texts = text.split(label)
            label_enc = self._tokenizer.encode(label)['input_ids']
            cls_enc = label_enc[0]
            sep_enc = label_enc[-1]
            label_enc = label_enc[1:-1]
            
            # 合并
            origin_enc = []
            label_ids = []
            for index, text in enumerate(texts):
                text_enc = self._tokenizer.encode(text)['input_ids']
                text_enc = text_enc[1:-1]
                origin_enc += text_enc
                label_ids += [label_list['O']] * len(text_enc)
                if index != len(texts) - 1:
                    origin_enc += label_enc
                    label_ids += [label_list['B']] + [label_list['I']] * (len(label_enc) - 1)

            origin_enc = [cls_enc] + origin_enc + [sep_enc]
            label_ids = [label_list['O']] + label_ids + [label_list['O']]
            
            # 截断
            if len(origin_enc) > self._max_len:
                origin_enc = origin_enc[:self._max_len-1] + origin_enc[-1:]
                label_ids = label_ids[:self._max_len-1] + label_ids[-1:]
            return np.array(origin_enc, dtype='int64'), np.array(label_ids, dtype='int64')


def batchify_fn(for_test=False):
    if for_test:
        return lambda samples, fn=Pad(axis=0, pad_val=tokenizer.pad_token_id): np.row_stack([data for data in fn(samples)])
    else:
        return lambda samples, fn=Tuple(Pad(axis=0, pad_val=tokenizer.pad_token_id),
                                        Pad(axis=0, pad_val=label_list['O'])): [data for data in fn(samples)]


def get_data_loader(data, tokenizer, batch_size=32, max_len=512, for_test=False):
    dataset = MyDataset(data, tokenizer, max_len, for_test)
    shuffle = True if not for_test else False
    data_loader = DataLoader(dataset=dataset, batch_size=batch_size, collate_fn=batchify_fn(for_test), shuffle=shuffle)
    return data_loader

### 3.3 模型搭建并进行训练
与之前不同的是模型换成了Token分类。由于Accuracy不再适用于Token分类，我们用Perplexity来大致衡量预测的准确度（接近1为最佳）。

In [6]:
import paddle
from paddle.static import InputSpec
from paddlenlp.metrics import Perplexity

# 模型和分词
model = SkepForTokenClassification.from_pretrained('skep_ernie_1.0_large_ch', num_classes=3)
tokenizer = SkepTokenizer.from_pretrained('skep_ernie_1.0_large_ch')

# 参数设置
data_name = 'cotedp'  # 更改此选项改变数据集

## 训练相关
epochs = 1
learning_rate = 2e-5
batch_size = 8
max_len = 512

## 数据相关
train_dataloader = get_data_loader(data_dict[data_name]['train'], tokenizer, batch_size, max_len, for_test=False)

input = InputSpec((-1, -1), dtype='int64', name='input')
label = InputSpec((-1, -1, 3), dtype='int64', name='label')
model = paddle.Model(model, [input], [label])

# 模型准备

optimizer = paddle.optimizer.Adam(learning_rate=learning_rate, parameters=model.parameters())
model.prepare(optimizer, loss=paddle.nn.CrossEntropyLoss(), metrics=[Perplexity()])

[2023-03-25 08:47:01,638] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/skep/skep_ernie_1.0_large_ch.pdparams and saved to /home/aistudio/.paddlenlp/models/skep_ernie_1.0_large_ch
[2023-03-25 08:47:01,642] [    INFO] - Downloading skep_ernie_1.0_large_ch.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/skep/skep_ernie_1.0_large_ch.pdparams
100%|██████████| 1238309/1238309 [00:17<00:00, 69415.54it/s]
W0325 08:47:19.674468    99 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0325 08:47:19.678896    99 device_context.cc:422] device: 0, cuDNN Version: 7.6.
[2023-03-25 08:47:27,725] [    INFO] - Downloading skep_ernie_1.0_large_ch.vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/skep/skep_ernie_1.0_large_ch.vocab.txt
100%|██████████| 55/55 [00:00<00:00, 32048.72it/s]


In [7]:
# 开始训练
model.fit(train_dataloader, batch_size=batch_size, epochs=epochs, save_freq=5, save_dir='./checkpoints', log_freq=200)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/1
step  200/3158 - loss: 0.0112 - Perplexity: 1.0547 - 302ms/step
step  400/3158 - loss: 0.0296 - Perplexity: 1.0339 - 304ms/step
step  600/3158 - loss: 0.0074 - Perplexity: 1.0274 - 304ms/step
step  800/3158 - loss: 0.0165 - Perplexity: 1.0240 - 300ms/step
step 1000/3158 - loss: 0.0042 - Perplexity: 1.0215 - 301ms/step
step 1200/3158 - loss: 0.0117 - Perplexity: 1.0198 - 300ms/step
step 1400/3158 - loss: 0.0101 - Perplexity: 1.0183 - 301ms/step
step 1600/3158 - loss: 0.0027 - Perplexity: 1.0174 - 301ms/step
step 1800/3158 - loss: 0.0143 - Perplexity: 1.0166 - 301ms/step
step 2000/3158 - loss: 0.0163 - Perplexity: 1.0159 - 301ms/step
step 2200/3158 - loss: 0.0147 - Perplexity: 1.0155 - 301ms/step
step 2400/3158 - loss: 0.0016 - Perplexity: 1.0149 - 301ms/step
step 2600/3158 - loss: 0.0078 - Perplexity: 1.0145 - 301ms/step
step 2800/3158 - loss: 0.0061 - Perplexity: 1.01

### 3.4 预测并保存

In [8]:
import re

# 导入预训练模型
checkpoint_path = './checkpoints/final'  # 填写预训练模型的保存路径

model = SkepForTokenClassification.from_pretrained('skep_ernie_1.0_large_ch', num_classes=3)
input = InputSpec((-1, -1), dtype='int64', name='input')
model = paddle.Model(model, [input])
model.load(checkpoint_path)

# 导入测试集
test_dataloader = get_data_loader(data_dict[data_name]['test'], tokenizer, batch_size, max_len, for_test=True)
# 预测保存

save_file = {'cotebd': './submission/COTE_BD.tsv', 'cotedp': './submission/COTE_DP.tsv', 'cotemfw': './submission/COTE_MFW.tsv'}
predicts = []
input_ids = []
for batch in test_dataloader:
    predict = model.predict_batch(batch)
    predicts += predict[0].argmax(axis=-1).tolist()
    input_ids += batch.numpy().tolist()

# 先找到B所在的位置，即标号为0的位置，然后顺着该位置一直找到所有的I，即标号为1，即为所得。
def find_entity(prediction, input_ids):
    entity = []
    entity_ids = []
    for index, idx in enumerate(prediction):
        if idx == label_list['B']:
            entity_ids = [input_ids[index]]
        elif idx == label_list['I']:
            if entity_ids:
                entity_ids.append(input_ids[index])
        elif idx == label_list['O']:
            if entity_ids:
                entity.append(''.join(tokenizer.convert_ids_to_tokens(entity_ids)))
                entity_ids = []
    return entity

with open(save_file[data_name], 'w', encoding='utf8') as f:
    f.write("index\tprediction\n")
    for idx, sample in enumerate(data_dict[data_name]['test']):
        qid = sample.split('\t')[0]
        entity = find_entity(predicts[idx], input_ids[idx])
        entity = list(set(entity))  # 去重
        entity = [re.sub('##', '', e) for e in entity]  # 去除英文编码时的特殊符号
        entity = [re.sub('[UNK]', '', e) for e in entity]  # 去除未知符号
        f.write(qid + '\t' + '\x01'.join(entity) + '\n')
    f.close()

[2023-03-25 09:10:29,069] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/skep_ernie_1.0_large_ch/skep_ernie_1.0_large_ch.pdparams


评价对象      预测标签
唐家三少      唐/B家/I三/I少/I，/O本/O名/O张/O威/O。/O
加勒	       加/B勒/I有/O用/O高/O大/O厚/O实/O的/O石/O头/O砌/O成/O的/O城/O墙/O。/O


